In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice'

In [14]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    metrics_file_name:Path 
    target_column:str
    all_params:dict

In [15]:
from AutoMobilePriceRegression.constants import *
from AutoMobilePriceRegression.utils.common import read_yaml, create_directories,save_json,load_json

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH) -> None:
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.GradientBoostingRegressor
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metrics_file_name=config.metrics_file_name,
            all_params=params,
            target_column=schema.name
        )

        return model_evaluation_config

In [17]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from urllib.parse import urlparse
from sklearn.metrics import r2_score,root_mean_squared_error
import joblib,json

In [18]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config= config

    def evaluation_metrics(self,actual,pred):
        RMSE=root_mean_squared_error(actual,pred) 
        return RMSE 

    def save_results(self):
        test_data=pd.read_csv(self.config.test_data_path)
        model=joblib.load(self.config.model_path)

        x_test,y_test=(
            test_data.drop(self.config.target_column,axis=1),
            test_data[self.config.target_column]
        )

        prediction=model.predict(x_test)

        (RMSE)=self.evaluation_metrics(y_test,prediction)

        scors={"RMSE":RMSE}

        save_json(path=Path(self.config.metrics_file_name),data=scors)         

In [19]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-09-07 00:53:34,379: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-07 00:53:34,381: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-07 00:53:34,385: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-09-07 00:53:34,387: INFO: common: created directory at: artifacts]
[2024-09-07 00:53:34,389: INFO: common: created directory at: artifacts/model_evaluation]
[2024-09-07 00:53:34,423: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
